In [4]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [5]:
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

100%|██████████| 26.4M/26.4M [00:01<00:00, 15.9MB/s]
100%|██████████| 29.5k/29.5k [00:00<00:00, 270kB/s]
100%|██████████| 4.42M/4.42M [00:00<00:00, 4.50MB/s]
100%|██████████| 5.15k/5.15k [00:00<00:00, 12.2MB/s]


In [6]:
batch_size = 64

# create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

# X: 이미지 데이터 묶음 (특징, feature)
# y: 정답 레이블 묶음 (레이블, lables)

for X, y in test_dataloader:
  print(f"Shape of X [N, C, H, W] : {X.shape}")
  print(f"Shape of y : {y.shape} {y.dtype}")

# 출력 결과 해석
# N: 배치 크기(Batch Size). 여기서는 batch_size = 64로 설정했기 때문에, 대부분의 배치에서 N은 64가 됩니다.

# C: 채널(Channels). 흑백 이미지인 패션 MNIST는 채널이 1입니다.

# H: 높이(Height). 패션 MNIST 이미지의 높이는 28픽셀입니다.

# W: 너비(Width). 패션 MNIST 이미지의 너비는 28픽셀입니다.

# 따라서 X.shape은 (64, 1, 28, 28) 이 됩니다.

# y.shape: X와 마찬가지로 배치 크기(64)와 같습니다. 즉, (64) 가 됩니다. 이는 각 이미지에 대한 하나의 정답 레이블이 있다는 것을 의미합니다.

# y.dtype: 레이블 데이터의 타입입니다. 일반적으로 torch.int64와 같은 정수형이 됩니다.

Shape of X [N, C, H, W] : torch.Size([64, 1, 28, 28])
Shape of y : torch.Size([64]) torch.int64
Shape of X [N, C, H, W] : torch.Size([64, 1, 28, 28])
Shape of y : torch.Size([64]) torch.int64
Shape of X [N, C, H, W] : torch.Size([64, 1, 28, 28])
Shape of y : torch.Size([64]) torch.int64
Shape of X [N, C, H, W] : torch.Size([64, 1, 28, 28])
Shape of y : torch.Size([64]) torch.int64
Shape of X [N, C, H, W] : torch.Size([64, 1, 28, 28])
Shape of y : torch.Size([64]) torch.int64
Shape of X [N, C, H, W] : torch.Size([64, 1, 28, 28])
Shape of y : torch.Size([64]) torch.int64
Shape of X [N, C, H, W] : torch.Size([64, 1, 28, 28])
Shape of y : torch.Size([64]) torch.int64
Shape of X [N, C, H, W] : torch.Size([64, 1, 28, 28])
Shape of y : torch.Size([64]) torch.int64
Shape of X [N, C, H, W] : torch.Size([64, 1, 28, 28])
Shape of y : torch.Size([64]) torch.int64
Shape of X [N, C, H, W] : torch.Size([64, 1, 28, 28])
Shape of y : torch.Size([64]) torch.int64
Shape of X [N, C, H, W] : torch.Size([64

In [10]:
# torch.accelerator.is_available(): 현재 시스템에 GPU가 사용 가능한지 확인합니다.
device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"
print(f"Using {device} device")

# Define model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),# 입력(784)을 받아 512개의 출력 노드를 가진 완전 연결(Fully Connected) 레이어를 만듭니다.
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

Using cpu device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [11]:
# optimizing the model parameters

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)
# 확률적(Stochastic) 경사 하강법 (SGD): 전체 데이터셋 중에서 무작위로 하나의 샘플을 뽑아 경사를 계산하고 가중치를 업데이트합니다. 이 방법은 학습이 빠르지만,
# 한 샘플의 경사가 전체 데이터의 경사를 잘 대표하지 못할 수 있어 학습이 불안정해질 수 있습니다.

In [19]:
def train(dataloader, model, loss_fn, optimizer):
  size = len(dataloader.dataset)
  model.train()
  for batch, (X, y) in enumerate(dataloader):
    X, y = X.to(device), y.to(device)
    # 이 줄은 입력 데이터 X와 정답 레이블 y를 앞서 설정한 하드웨어 장치(GPU 또는 CPU로 옮깁니다. 이는 연산을 해당 장치에서 처리하기 위함입니다.
    # Compute prediction error
    pred = model(X)
    loss = loss_fn(pred, y)

    # Backpropagation
    loss.backward()
    optimizer.step() # 경사를 사용하여 모델의 가중치를 업데이트 하도록함
    optimizer.zero_grad() # 경사를 0으로 초기화, 파이토치에서는 경사가 누적되기 때문에, 다음 배치를 처리하기 전에 이전 배치의 경사 값을 초기화해주어야 올바른 학습이 가능하고, 매우 중요한 작업이다.

    if batch % 100 == 0:
          loss, current = loss.item(), (batch + 1) * len(X)
          print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [24]:
def test(dataloader, model, loss_fn):
  size = len(dataloader.dataset)
  num_batches = len(dataloader)
  model.eval() # 평가모드로 전환
  test_loss, correct = 0, 0
  with torch.no_grad(): # 경사계싼을 비활성화 -> 테스트할 때는 필요 없어서
    for X, y in dataloader:
      X, y = X.to(device), y.to(device)
      pred = model(X)  # 예측
      test_loss += loss_fn(pred, y).item() # 모델의 예측 pred와 실제 정답 y 사이의 손실(loss)을 계산합니다. item() 메서드는 텐서(Tensor)에 저장된 값을 Python 숫자로 변환합니다. 모든 배치의 손실 값을 누적하여 전체 데이터셋의 평균 손실을 계산하는 데 사용
      correct += (pred.argmax(1)==y).type(torch.float).sum().item() # 모델이 정확하게 예측한 샘플의 수 계산
    test_loss /= num_batches
    correct /= size # 전체 테스트 데이터에 대한 평균 손실과 정확도를 계산합니다. num_batches는 전체 배치의 수, size는 전체 데이터셋의 크기
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [25]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 1.896248  [   64/60000]
loss: 1.863762  [ 6464/60000]
loss: 1.749194  [12864/60000]
loss: 1.793510  [19264/60000]
loss: 1.685166  [25664/60000]
loss: 1.619543  [32064/60000]
loss: 1.649456  [38464/60000]
loss: 1.545843  [44864/60000]
loss: 1.573943  [51264/60000]
loss: 1.463207  [57664/60000]
Test Error: 
 Accuracy: 63.0%, Avg loss: 1.491402 

Epoch 2
-------------------------------
loss: 1.553756  [   64/60000]
loss: 1.516045  [ 6464/60000]
loss: 1.367917  [12864/60000]
loss: 1.444996  [19264/60000]
loss: 1.328352  [25664/60000]
loss: 1.314213  [32064/60000]
loss: 1.340699  [38464/60000]
loss: 1.258010  [44864/60000]
loss: 1.299067  [51264/60000]
loss: 1.200658  [57664/60000]
Test Error: 
 Accuracy: 63.9%, Avg loss: 1.227423 

Epoch 3
-------------------------------
loss: 1.299771  [   64/60000]
loss: 1.277559  [ 6464/60000]
loss: 1.115266  [12864/60000]
loss: 1.225175  [19264/60000]
loss: 1.103066  [25664/60000]
loss: 1.119648  [32064/600

In [26]:
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth


In [27]:
model = NeuralNetwork().to(device)
model.load_state_dict(torch.load("model.pth", weights_only=True))

<All keys matched successfully>

In [32]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    x = x.to(device)
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Ankle boot", Actual: "Ankle boot"
